#### Fine-tuning LLM project 
\
\
Uses LLAMA from Meta
\
A basic finetuning project, to try out finetuning, and to learn the basics of finetuning through Python SDK

In [1]:
import os
import lamini
from lamini import Lamini

import datasets #this library allows easy access to datasets on HuggingFace Datasets Hub
import jsonlines
import pandas as pd
import itertools
from pprint import pprint
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM

lamini.api_key = "ae666640b62327d2dff4c6e147cc34c6c0cecb0b8e7887ffe49a795104980587"

c:\Users\cwong2\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First we will look at the llama model from meta. We will look at both their regular llm, and llm that has been trained to be a chatbot. We will be looking at LLama-2-7b-hf (https://huggingface.co/meta-llama/Llama-2-7b-hf) and LLama-2-7b-chat-hf which are the fine tuned models that are optimized for chat use cases. Later we will look at also look at Meta-LLama-3.1-8B-Instruct (https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

In [2]:
nf_llm = Lamini(model_name="meta-llama/Llama-2-7b-hf")

In [3]:
print(nf_llm.generate("Who are the best NBA players in the league right now?"))


Who are the best NBA players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now?
The NBA is a league full of superstars, but who are the best players in the league right now?
The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now?
The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now? The NBA is a league full of superstars, but who are the best players in the league right now? The N

Lets take a look at the fine tuned output now - these have just been fine-tuned by meta themselves. 

In [4]:
f_llm = Lamini(model_name="meta-llama/Llama-2-7b-chat-hf")
print(f_llm.generate("Who are the best NBA players in the league right now?"))



The NBA is home to some of the most talented athletes in the world, and there are several players who are currently dominating the league. Here are some of the best NBA players in the league right now:

1. Luka Doncic - Dallas Mavericks: Doncic is a 22-year-old phenom who is leading the NBA in scoring, averaging 31.7 points, 9.7 rebounds, and 8.7 assists per game. He is a versatile player who can dominate games on both ends of the court.
2. Kevin Durant - Brooklyn Nets: Durant is a 32-year-old forward who is known for his scoring ability and athleticism. He is averaging 26.9 points, 7.1 rebounds, and 5.9 assists per game, and is a four-time NBA scoring champion.
3. Giannis Antetokounmpo - Milwaukee Bucks: Antetokounmpo is a 26-year-old forward who is one of the most dominant players in the league. He is averaging 29.6 points, 12.7 rebounds, and 5.9 assists per game, and is a two-time NBA Most Valuable Player.
4. Stephen Curry - Golden State Warriors: Curry is a 33-year-old guard who 

In [5]:
print(f_llm.generate("I am a basketball player who wants to get in shape. Write a 7 day workout plan for me that would give me enough rest, while also improving my cardio, taking into account that I play two games a week on Tuesday and Wednesday."))



Here is a 7-day workout plan for you as a basketball player who wants to get in shape:

Day 1: Cardio and Strength Training

* Warm-up: 5-10 minutes of light cardio (jogging, jumping jacks, etc.)
* Strength training: 3 sets of 10-15 reps of exercises like squats, lunges, push-ups, and rows
* Cardio: 30 minutes of steady-state cardio (jogging, cycling, etc.)
* Cool-down: 5-10 minutes of stretching

Day 2: Rest day

Day 3: Plyometric Training

* Warm-up: 5-10 minutes of light cardio
* Plyometric training: 3 sets of 10-15 reps of exercises like box jumps, depth jumps, and burpees
* Cardio: 30 minutes of steady-state cardio
* Cool-down: 5-10 minutes of stretching

Day 4: Strength Training

* Warm-up: 5-10 minutes of light cardio
* Strength training: 3 sets of 10-15 reps of exercises like deadlifts, bench press, and rows
* Cardio: 30 minutes of steady-state cardio
* Cool-down: 5-10 minutes of stretching

Day 5: Rest day

Day 6: Agility Training

* Warm-up: 5-10 minutes of light cardio
* A

In [6]:
print(f_llm.generate("""Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent:"""))

I see. Can you please provide me with your order number so I can look into this for you?
Customer: I don't have the order number.
Agent: Okay, no worries. Can you please tell me the name of the item you didn't receive?
Customer: It was a blue blanket.
Agent: I see. I'm going to check on the status of your order for you. Can you please hold for just a moment?
Customer: Okay, thank you.
Agent: (pause) I apologize, but it looks like your order was delivered to the wrong address. Can you please provide me with your correct address so I can assist you in getting the blanket delivered to you?
Customer: (sighs) I don't know why this is happening to me. I've been ordering from Amazon for years and never had any problems.
Agent: I apologize for any inconvenience this has caused. I'm here to help you resolve the issue as quickly as possible. Can you please provide me with your correct address so I can assist you?
Customer: (frustrated) Fine. It's 123 Main St, Apartment 302. Can you please fix th

We will use a pretrained dataset and compare it with a finetuning dataset (taken from HuggingFace) - we will use streaming=True as the dataset is huge, so we don't want to load the entire dataset in at once, so we stream one line in at a time.

In [7]:
pretrained_dataset = load_dataset("c4", "en", split="train", streaming=True, trust_remote_code=True)

C:\Users\cwong2\.cache\huggingface\modules\datasets_modules\datasets\c4\584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db\c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


In [8]:
n=5
top_n = itertools.islice(pretrained_dataset, n)
for i in top_n:
    print(i)


{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.', 'timestamp': '2019-04-25T12:57:54Z', 'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-missoula/'}
{'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240g

In [9]:
instruction_data_df = pd.read_json("lamini_docs.jsonl", lines=True)
instruction_data_df.head(5)

,question,answer
0,What are the different types of documents avai...,"Lamini has documentation on Getting Started, A..."
1,What is the recommended way to set up and conf...,Lamini can be downloaded as a python package a...
2,How can I find the specific documentation I ne...,"You can ask this model about documentation, wh..."
3,Does the documentation include explanations of...,Our documentation provides both real-world and...
4,Does the documentation provide information abo...,External dependencies and libraries are all av...


In [10]:
# Putting the dataset into a dictionary 
examples = instruction_data_df.to_dict()
# Taking a look at the first element in the dictionary, formatting into a line of text by concatenation
text = examples["question"][0] + examples["answer"][0]
text

"What are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?Lamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/."

In [11]:
# Other ways of formatting
if "question" in examples and "answer" in examples:
    text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
    text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
    text = examples["input"][0] + examples["output"][0]
else:
    text = examples["text"][0]

# We want to give more structure, so that it helps the model 
# Writing a prompt template to hold both the questions and answers - the ### delimiters are used to help the model 
prompt_template_qa = """### Question: 
{question}

### Answer:
{answer}"""

In [12]:
question = examples["question"][0]
answer = examples["answer"][0]

text_with_prompt_template = prompt_template_qa.format(question = question, answer = answer)
text_with_prompt_template

"### Question: \nWhat are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?\n\n### Answer:\nLamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/."

In [13]:
# Writing a prompt template to hold just the question - helps to have this for evaluation
prompt_template_q = """### Question:
{question}

### Answer:"""

In [14]:
prompt_template_q.format(question=question, answer=answer)

"### Question:\nWhat are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?\n\n### Answer:"

In [15]:
# Applying the template to the entire dataset
num_examples = len(examples["question"])
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []
for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]

    text_with_prompt_template_qa = prompt_template_qa.format(question=question, answer=answer)
    finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})

    text_with_prompt_template_q = prompt_template_q.format(question=question)
    finetuning_dataset_question_answer.append({"question": text_with_prompt_template_q, "answer": answer})

In [16]:
pprint(finetuning_dataset_text_only[0])
pprint(finetuning_dataset_question_answer[0])

{'text': '### Question: \n'
         'What are the different types of documents available in the '
         "repository (e.g., installation guide, API documentation, developer's "
         'guide)?\n'
         '\n'
         '### Answer:\n'
         'Lamini has documentation on Getting Started, Authentication, '
         'Question Answer Model, Python Library, Batching, Error Handling, '
         'Advanced topics, and class documentation on LLM Engine available at '
         'https://lamini-ai.github.io/.'}
{'answer': 'Lamini has documentation on Getting Started, Authentication, '
           'Question Answer Model, Python Library, Batching, Error Handling, '
           'Advanced topics, and class documentation on LLM Engine available '
           'at https://lamini-ai.github.io/.',
 'question': '### Question:\n'
             'What are the different types of documents available in the '
             'repository (e.g., installation guide, API documentation, '
             "developer's gui

In [17]:
# Saving new dataset into a new file - jsonlines is the most common (each line )
with jsonlines.open(f'lamini_docs_processed.jsonl', "w") as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [18]:
# Saving lamini dataset from hugging face - you can also upload your dataset to huggingface and then load it using load_dataset
finetuning_dataset_name = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_name)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


Instruction fine-tuning: a type of fine tuning that makes an LLM more like a chatbot, and allows it to have conversations. There are also other types of fine-tuning: agentic, copilot (for helping write code), reasoning, routing

In [19]:
# We will use a fine tuning dataset from Alpaca
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split ="train", streaming=True)

In [20]:
# Taking a look at a few examples from the dataset
m = 5
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(instruction_tuned_dataset, m))
for j in top_m:
  print(j)

Instruction-tuned dataset:
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three p

In [21]:
# top_m is a list of dictionaries, with all the instructions, input, output, and text in the fine tuning dataset 
top_m

[{'instruction': 'Give three tips for staying healthy.',
  'input': '',
  'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
  'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'},
 {'instruction': 'What are the three primary colors?',
  'input': '',
  'output': 'The three primary colors are red, blue, and yellow.',
  'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary color

In [22]:
# Prompt templates - this was used in the Alpaca dataset for the purpose of fine-tuning
prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""

prompt_template_without_input = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [23]:
# Adding data to the prompts 
processed_data = []
for j in top_m:
    if not j["input"]: #if input is empty
        processed_prompt = prompt_template_without_input.format(instruction=j["instruction"])
    else:
        processed_prompt = prompt_template_with_input.format(instructions=j["instruction"], input=j["input"])

    processed_data.append({"input": processed_prompt, "output": j["output"]})

In [24]:
pprint(processed_data[0])

{'input': 'Below is an instruction that describes a task. Write a response '
          'that appropriately completes the request.\n'
          '\n'
          '### Instruction:\n'
          'Give three tips for staying healthy.\n'
          '\n'
          '### Response:',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [25]:
# Saving data 
with jsonlines.open(f'alpaca_processed.jsonl', "w") as writer:
    writer.write_all(processed_data)

In [26]:
# Note that this current dataset, with the prompt template, has been loaded onto HuggingFace already. We can access with:

dataset_path_hf = "lamini/alpaca"
dataset_hf = load_dataset(dataset_path_hf)
print(dataset_hf)

# We could also upload our own to hugging face 

# !pip install huggingface_hub
# !huggingface-cli login

# import pandas as pd
# import datasets
# from datasets import Dataset

# finetuning_dataset = Dataset.from_pandas(pd.DataFrame(data=finetuning_dataset))
# finetuning_dataset.push_to_hub(dataset_path_hf)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


We can try some smaller models - will use a 70 million param model in this case, but something like llama-7 has 7 billion params which is much larger, and something like gpt-3 has closer to 70 billion params\
\
The from_pretrained() method is used to load pre-trained models or tokenizers so you can use base LLMs and further fine tune for your own use cases.\
\
The model pythia-70m has not been instruction fine-tuned, and will be the model we will be finetuning ourselves

In [27]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m") 
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [28]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
    # Tokenize
    input_ids = tokenizer.encode(
        text,
        return_tensors = "pt",
        truncation = True,
        max_length=max_input_tokens
    )

    # Generate 
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length=max_output_tokens
    )

    # Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

    # Strip the prompt 
    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer

In [29]:
test_sample = finetuning_dataset["test"][0]
print(test_sample)

{'question': 'Can Lamini generate technical documentation or user manuals for software projects?', 'answer': 'Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.', 'input_ids': [5804, 418, 4988, 74, 6635, 7681, 10097, 390, 2608, 11595, 84, 323, 3694, 6493, 32, 4374, 13, 418, 4988, 74, 476, 6635, 7681, 10097, 285, 2608, 11595, 84, 323, 3694, 6493, 15, 733, 4648, 3626, 3448, 5978, 5609, 281, 2794, 2590, 285, 44003, 10097, 326, 310, 3477, 281, 2096, 323, 1097, 7681, 285, 1327, 14, 48746, 4212, 15, 831, 476, 5321, 12259, 247, 1534, 2408, 273, 673, 285, 3434, 275, 6153, 10097, 13, 6941, 731, 281, 2770, 327, 643, 7794, 273, 616, 6493, 15], 'attention

In [30]:
print(test_sample["question"])

Can Lamini generate technical documentation or user manuals for software projects?


In [31]:
# Lets try this pythia-70m model that has not yet been finetuned and see what kind of output it gives us
inference(test_sample["question"], model, tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'\n\nI have a question about the following:\n\nHow do I get the correct documentation to work?\n\nA:\n\nI think you need to use the following code:\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following code to get the correct documentation.\n\nA:\n\nYou can use the following'

Clearly, the model output is not very good. It is in english, and seems to have picked up the word "documentation", but that is about the extent of its understanding. Lets take a look at what a fine tuned model using the same pythia-70m LLM as a base model looks like. \
\
You can find the model on Hugging Face \
\
lamini/lamini_docs_finetuned is a model that is on HuggingFace (https://huggingface.co/lamini/lamini_docs_finetuned)

In [32]:
instruction_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
inference(test_sample["question"], model=instruction_model, tokenizer=tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini'

We will fine tune this pythia-70m model ourselves, and train it so that it can learn the context of the lamini documentation and answer questions.\
\
\
There are steps for preparing data for finetuning a LLM:
1. Collect instruction-response pairs
2. Concatenate pairs, and add prompt templates if required
3. Tokenize - pad and truncate
4. Split data into test and train
\
\
Lets first take a look at our tokenizer that we have defined. We will look at some toy examples first, before we go on and work on the dataset. We used tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m") 

In [33]:
text = "Hi, how are you?"
tokenizer(text)

{'input_ids': [12764, 13, 849, 403, 368, 32], 'attention_mask': [1, 1, 1, 1, 1, 1]}

Each LLM is associated with a tokenizer --> giving the wrong tokenizer will confuse the model, so you will have to make sure you use the correct tokenizer. AutoTokenizer class from transformers will automatically help you tokenize your model. 

In [34]:
encoded_text = tokenizer(text)["input_ids"]
encoded_text

[12764, 13, 849, 403, 368, 32]

In [35]:
decoded_text = tokenizer.decode(encoded_text)
decoded_text

'Hi, how are you?'

In [36]:
# Tokenizing multiple texts
list_texts = ["Hi, how are you?", "I'm good", "Yes"]
tokenizer(list_texts)

{'input_ids': [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175], [4374]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1], [1]]}

Everything in a batch should be the same length - so they should have the same length for input_ids. We will use padding. 

In [37]:
encoded_texts = tokenizer(list_texts)
print("Encoded several texts: ", encoded_texts["input_ids"])

Encoded several texts:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175], [4374]]


In [38]:
# Padding - ensure input sequences are of the same length 
# eos_token marks the end of sequence token
tokenizer.pad_token = tokenizer.eos_token
encoded_texts_longest = tokenizer(list_texts, padding = True)
print("Using padding: ", encoded_texts_longest["input_ids"])

Using padding:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175, 0, 0, 0], [4374, 0, 0, 0, 0, 0]]


max_length is the maximum length it can handle and take in. There is a limit to the length. Truncation makes the longer, encoded texts shorter, so that it can fit into the model. In the below code, the max length has been set to 3, and the truncation has been set to true. This means that only the first 3 input_ids will shown if there are more than 3 input_ids. The default is right side truncation

In [39]:
# Truncation
encoded_texts_truncation = tokenizer(list_texts, max_length=3, truncation=True)
encoded_texts_truncation["input_ids"]

[[12764, 13, 849], [42, 1353, 1175], [4374]]

In [40]:
# Using left side truncation
tokenizer.truncation_side = "left"
encoded_texts_truncation_left = tokenizer(list_texts, max_length=3, truncation=True)
encoded_texts_truncation_left["input_ids"]

[[403, 368, 32], [42, 1353, 1175], [4374]]

In [41]:
# Padding and truncation
encoded_texts_both = tokenizer(list_texts, max_length=3, truncation=True, padding=True)
encoded_texts_both["input_ids"]

[[403, 368, 32], [42, 1353, 1175], [4374, 0, 0]]

Now lets move on to our dataset, and prepare our instruction dataset for finetuning. Preparing our dataset - this will be the instruction dataset

In [42]:
# loading the dataset files
filename = "lamini_docs.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
examples = instruction_dataset_df.to_dict()

In [43]:
# In the instruction dataset, there are some that are question-answer, instruction-response, and input-output
if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]

In [44]:
# "hydrating the prompts" - putting the data into the prompt templates
prompt_template = """### Question:
{question}

### Answer:"""

num_examples = len(examples["question"])
finetuning_dataset = []
for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]
    text_with_prompt_template = prompt_template.format(question=question)
    finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})


In [45]:
# Taking a look at the first 5 data points in the dataset of question and answer
finetuning_dataset[0:5]

[{'question': "### Question:\nWhat are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?\n\n### Answer:",
  'answer': 'Lamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/.'},
 {'question': '### Question:\nWhat is the recommended way to set up and configure the code repository?\n\n### Answer:',
  'answer': 'Lamini can be downloaded as a python package and used in any codebase that uses python. Additionally, we provide a language agnostic REST API. We’ve seen users develop and train models in a notebook environment, and then switch over to a REST API to integrate with their production environment.'},
 {'question': '### Question:\nHow can I find the specific documentation I need for a particular feature or function?\n\n### Answer:',
  'answer': 

In [46]:
# Tokenizing a single example - here we return the tensors as a numpy array for simplicity
text = finetuning_dataset[0]["question"] + finetuning_dataset[0]["answer"]
tokenized_inputs = tokenizer(text, return_tensors = "np", padding=True)
tokenized_inputs["input_ids"]

array([[ 4118, 19782,    27,   187,  1276,   403,   253,  1027,  3510,
          273,  7177,  2130,   275,   253, 18491,   313,    70,    15,
           72,   904, 12692,  7102,    13,  8990, 10097,    13, 13722,
          434,  7102,  6177,   187,   187,  4118, 37741,    27,    45,
         4988,    74,   556, 10097,   327, 27669, 11075,   264,    13,
         5271, 23058,    13, 19782, 37741, 10031,    13, 13814, 11397,
           13,   378, 16464,    13, 11759, 10535,  1981,    13, 21798,
        12989,    13,   285,   966, 10097,   327, 21708,    46, 10797,
         2130,   387,  5987,  1358,    77,  4988,    74,    14,  2284,
           15,  7280,    15,   900, 14206]])

In [47]:
max = 2048
max_length = min(tokenized_inputs["input_ids"].shape[1], max)
tokenized_inputs = tokenizer(text, return_tensors="np", truncation=True, max_length=max_length)
tokenized_inputs["input_ids"]

array([[ 4118, 19782,    27,   187,  1276,   403,   253,  1027,  3510,
          273,  7177,  2130,   275,   253, 18491,   313,    70,    15,
           72,   904, 12692,  7102,    13,  8990, 10097,    13, 13722,
          434,  7102,  6177,   187,   187,  4118, 37741,    27,    45,
         4988,    74,   556, 10097,   327, 27669, 11075,   264,    13,
         5271, 23058,    13, 19782, 37741, 10031,    13, 13814, 11397,
           13,   378, 16464,    13, 11759, 10535,  1981,    13, 21798,
        12989,    13,   285,   966, 10097,   327, 21708,    46, 10797,
         2130,   387,  5987,  1358,    77,  4988,    74,    14,  2284,
           15,  7280,    15,   900, 14206]])

In [48]:
# We want to make the above process into a function so we can run it on the entire dataset
def tokenize_function(examples):
   if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
   elif "instruction" in examples and "response" in examples:
      text = examples["instruction"][0] + examples["response"][0]
   elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
   else:
      text = examples["text"][0]
        
   tokenizer.pad_token = tokenizer.eos_token
   tokenized_inputs = tokenizer(text, return_tensors="np", padding=True)
   max = 2048
   max_length = min(tokenized_inputs["input_ids"].shape[1], max)
   tokenizer.truncation_side = "left"
   tokenized_inputs = tokenizer(text, return_tensors="np", truncation=True, max_length=max_length)

   return tokenized_inputs

In [49]:
finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")
tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched = True,
    batch_size = 1,
    drop_last_batch = True
)

print(tokenized_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1400
})


We need to add the "labels" column to split the dataset into test and train. 

In [50]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [51]:
# Notice that all the datasets have question, answer, input_ids, attention_mask, and labels columns
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


The above steps were just manually done on a dataset, so that we can go through the process of what it would be like preparing a dataset for training LLM models. We used a dataset that was already available on huggingface. The exact same dataset can be retrieved from hugging face directly below: 

In [52]:
# Trying out some datasets
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = datasets.load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


There are other datasets we can also take a look at

In [53]:
taylor_swift_dataset = "lamini/taylor_swift"
bts_dataset = "lamini/bts"
open_llms_dataset = "lamini/open_llms"


In [54]:
dataset_taylor_swift = datasets.load_dataset(taylor_swift_dataset)
print(dataset_taylor_swift["train"][0])

{'question': "What is the controversy surrounding Taylor Swift's music and how has it impacted her career?", 'answer': 'Taylor Swift has been involved in several controversies throughout her career, including her feud with Kanye West and Kim Kardashian, her lawsuit against a radio DJ who allegedly groped her, and her recent feud with Scooter Braun. These controversies have impacted her career in several ways. First, they have made her a more polarizing figure in the music industry, with some fans supporting her and others criticizing her. Second, they have led to a decrease in her popularity among some listeners, particularly those who do not agree with her political views or her actions in the feuds. Finally, they have led to a decrease of her music being played on some radio stations, which has impacted her ability to reach new audiences', 'input_ids': [1276, 310, 253, 16305, 8704, 11276, 24619, 434, 3440, 285, 849, 556, 352, 27857, 617, 5249, 32, 37979, 24619, 556, 644, 3206, 275, 2

In [55]:
print(dataset_taylor_swift)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 783
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 87
    })
})


In [56]:
dataset_open_llms = datasets.load_dataset(open_llms_dataset)
print(dataset_open_llms["train"][0])

{'question': 'AlekseyKorshuk-chatml-pyg-v1: AlekseyKorshuk-chatml-pyg-v1: AlekseyKorshuk-chatml-pyg-v1: What dataset was used for training?', 'answer': 'The None dataset was used for training.', 'input_ids': [44163, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72, 14, 87, 18, 27, 16660, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72, 14, 87, 18, 27, 16660, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72, 14, 87, 18, 27, 1737, 10895, 369, 908, 323, 3733, 32, 510, 8256, 10895, 369, 908, 323, 3733, 15], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [44163, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72, 14, 87, 18, 27, 16660, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72, 14, 87, 18, 27, 16660, 76, 5462, 44, 641, 73, 2788, 14, 23481, 1686, 14, 4789, 72

In [57]:
print(dataset_open_llms)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 112
    })
})


## Model Training 
This next section focuses on model training. We want to take a look at how the code is running under the hood


In [ ]:
# model = Lamini("EleutherAI/pythia-70m") 
# model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")
# model.train(is_public=True) 

AttributeError: 'Lamini' object has no attribute 'load_data_from_jsonlines'

In [67]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)